## TD 4
### Corally Ngov (2392143) et Lina Sadat (2378349)
---


### i)

In [47]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from scipy.optimize import fsolve
from scipy.integrate import solve_ivp

In [48]:
def bilans(t, x, p, u):
    # Assignation des variables d’etat
    [Ca, Cb, Cc, T] = x

    # Assignation des parametres
    [rho, Cp, V, A1, A2, E1R, E2R, dH1, dH2, Tc, UA] = p

    # Assignation des variables manipulees
    [F, Cain, Tin, Tc] = u

    k1 = A1 * np.exp(-E1R / T)
    k2 = A2 * np.exp(-E2R / T)
    # Systeme d’equations differentielles
    dCadt = (F / V) * (Cain - Ca) - (k1 * Ca)
    dCbdt = (-F / V) * Cb + (k1 * Ca) - (k2 * Cb)
    dCcdt = (-F / V) * Cc + (k2 * Cb)
    dTdt = (F / V) * (Tin - T) - (1 / (rho * Cp)) * ((k1 * Ca * dH1) + (k2 * Cb * dH2)) - (UA / (rho * V * Cp)) * (T - Tc)
    return [dCadt, dCbdt, dCcdt, dTdt]

#Conditions initiales
p = [1.0, 4.18, 60, 2.53e9, 7.34e10, 7500, 9500, -71.9, 33.1, 313, 85]
t_sim = [0, 100] #en min
u0 = [10, 7.5, 293, 313]
x0=[2, 0, 0, 293]
x0 = fsolve(lambda x, p, u: bilans(0, x, p, u), x0, args=(p, u0))

sim = solve_ivp(bilans, t_sim, x0, method = 'RK45', args = (p, u0), max_step = 0.01)

Ca, Cb, Cc, T = sim.y

#Valeurs en regime permanent
print('Valeurs en regime permanent :')
print(f'CA = {Ca[-1]:.2f} mol/L')
print(f'CB = {Cb[-1]:.2f} mol/L')
print(f'CC = {Cc[-1]:.2f} mol/L')
print(f'T = {T[-1]:.2f} K')




Valeurs en regime permanent :
CA = 1.88 mol/L
CB = 4.59 mol/L
CC = 1.03 mol/L
T = 335.59 K


In [49]:
#Fichier script permettant d’´evaluer les matrices A et B
#aux conditions de regime permanent
#Definition des variables symboliques
Ca, Cb, Cc, T = sp.symbols('Ca Cb Cc T')
F, Cain, Tin, Tc = sp.symbols('F Cain Tin Tc')
rho, Cp, V, A1, A2, E1R, E2R, dH1, dH2, UA = sp.symbols('rho Cp V A1 A2 E1R E2R dH1 dH2 UA')

k1 = A1*sp.exp(-E1R/T)
k2 = A2*sp.exp(-E2R/T)

#Systeme d’equations differentielles
dCadt = (F/V)*(Cain-Ca)- (k1*Ca)
dCbdt = (-F/V)*Cb + (k1*Ca)- (k2*Cb)
dCcdt = (-F/V)*Cc + (k2*Cb)
dTdt = (F/V)*(Tin-T)- (1/(rho*Cp))*((k1*Ca*dH1) + (k2*Cb*dH2))- (UA/(rho*V*Cp))*(T-Tc)

f = sp.Matrix([dCadt, dCbdt, dCcdt, dTdt])
x = sp.Matrix([Ca, Cb, Cc, T])
u = sp.Matrix([F, Tin, Cain, Tc])

#Calcul symbolique des matrices A et B
mat_A = sp.simplify(f.jacobian(x))
mat_B = sp.simplify(f.jacobian(u))

print("Matrice A- symbolique =")
sp.pprint(mat_A)

print("Matrice B- symbolique =")
sp.pprint(mat_B)

#Calcul numerique des matrices A et B
#Entrez ici vos valeurs des variables d’etat en regime permanent obtenues a l’aide de la fonction fsolve a la question i).
valeurs_num = {
 Ca: Ca, Cb: Cb, Cc: Cc, T: T,
 F: 10.0, Cain: 7.5, Tin: 293, Tc: 313,
 rho: 1.0, Cp: 4.18, V: 60,
 A1: 2.53*10**9, A2: 7.34*10**10, E1R: 7500, E2R: 9500,
 dH1:-71.9, dH2: 33.1, UA: 85
 }

mat_A_num = mat_A.subs(valeurs_num).evalf()
mat_B_num = mat_B.subs(valeurs_num).evalf()

print("Matrice A- num´erique =")
sp.pprint(mat_A_num)

print("Matrice B- num´erique =")
sp.pprint(mat_B_num)

Matrice A- symbolique =
⎡                                                                       -E1R   ↪
⎢      -E1R                                                             ─────  ↪
⎢      ─────                                                              T    ↪
⎢        T     F                                            -A₁⋅Ca⋅E1R⋅ℯ       ↪
⎢- A₁⋅ℯ      - ─         0          0                       ────────────────── ↪
⎢              V                                                     2         ↪
⎢                                                                   T          ↪
⎢                                                                              ↪
⎢                                                             -E1R             ↪
⎢       -E1R            -E2R                                  ─────            ↪
⎢       ─────           ─────                                   T              ↪
⎢         T               T     F                  A₁⋅Ca⋅E1R⋅ℯ        A₂⋅Cb⋅E2 ↪
⎢   

### ii)

In [50]:
valeurs_propres = mat_A.eigenvals()
print("Valeurs propres de A = ")
sp.pprint(valeurs_propres)


Valeurs propres de A = 
⎧                                                                              ↪
⎪                                                                              ↪
⎪                                                                              ↪
⎪                                                                              ↪
⎪                                                                              ↪
⎪                                                                              ↪
⎪                                                                              ↪
⎪                                                                              ↪
⎪                                                                              ↪
⎪                                                                              ↪
⎪                         -E1R        -E1R                   -E2R        -E2R  ↪
⎪                         ─────       ─────                  ─────       ───── ↪
⎪   